#### NOTA: En este cuaderno se crearan las Tablas `venta_header` y `venta_detail` del Datawarehouse

***
### Importando Librerias

In [1]:
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql.functions import col, count, when, isnull

***
Creando Session de Spark

In [3]:
spark = SparkSession.builder.appName('venta_header-venta_detail').getOrCreate()

***
Importando Conexion a Contenedor `mod4_postgres`

In [4]:
from db.connection import connec_db

***
Trabajando el csv asociado a  `compra`

In [4]:
venta = spark.read.csv("Data/Venta.csv", sep=",", header=True, inferSchema=True)

***
Visualizando como vienen los Datos

In [5]:
venta.show(5)

+-------+----------+-------------+-------+---------+----------+----------+----------+------+--------+
|IdVenta|     Fecha|Fecha_Entrega|IdCanal|IdCliente|IdSucursal|IdEmpleado|IdProducto|Precio|Cantidad|
+-------+----------+-------------+-------+---------+----------+----------+----------+------+--------+
|      1|2018-03-09|   2018-03-17|      3|      969|        13|      1674|     42817|813.12|       2|
|      2|2018-12-28|   2018-12-29|      2|      884|        13|      1674|     42795|543.18|       3|
|      3|2016-03-28|   2016-03-31|      2|     1722|        13|      1674|     42837|430.32|       1|
|      4|2017-10-23|   2017-10-24|      3|     2876|        13|      1674|     42834|818.84|       2|
|      5|2017-11-22|   2017-11-25|      2|      678|        13|      1674|     42825|554.18|       3|
+-------+----------+-------------+-------+---------+----------+----------+----------+------+--------+
only showing top 5 rows



***
Visualizando la Inferencia del Schema

In [6]:
venta.printSchema()

root
 |-- IdVenta: integer (nullable = true)
 |-- Fecha: date (nullable = true)
 |-- Fecha_Entrega: date (nullable = true)
 |-- IdCanal: integer (nullable = true)
 |-- IdCliente: integer (nullable = true)
 |-- IdSucursal: integer (nullable = true)
 |-- IdEmpleado: integer (nullable = true)
 |-- IdProducto: integer (nullable = true)
 |-- Precio: double (nullable = true)
 |-- Cantidad: integer (nullable = true)



***
Buscando Valores Faltantes

In [7]:
# Contar valores nulos por columna
venta.agg(*[count(when(isnull(c), c)).alias(c) for c in venta.columns]).show()



+-------+-----+-------------+-------+---------+----------+----------+----------+------+--------+
|IdVenta|Fecha|Fecha_Entrega|IdCanal|IdCliente|IdSucursal|IdEmpleado|IdProducto|Precio|Cantidad|
+-------+-----+-------------+-------+---------+----------+----------+----------+------+--------+
|      0|    0|            0|      0|        0|         0|         0|         0|   924|     892|
+-------+-----+-------------+-------+---------+----------+----------+----------+------+--------+



En las cantidades Faltantes segun los Homeworks se asumio colocar 1 <br>
En cuanto a los precios se localizaron todos en la tabla `producto` y se buscan por 'IdProducto`


In [8]:
# Se asigna 1 a las cantidades
venta = venta.fillna({'Cantidad':1})

# Se coloca 0 por ahora a los precios
venta = venta.fillna({'Precio':0})

***
Hay que obtener la informacion de los precios de la data de Productos

In [9]:
producto = spark.read.csv("Data/PRODUCTOS.csv", sep=",", header=True, inferSchema=True)

In [10]:
producto = producto.withColumnRenamed("Precio","precio_p")

In [11]:
producto.show(1)

+-----------+------------------+---------+--------+
|ID_PRODUCTO|          Concepto|     Tipo|precio_p|
+-----------+------------------+---------+--------+
|      42737|EPSON COPYFAX 2000|IMPRESIÓN|  1658.0|
+-----------+------------------+---------+--------+
only showing top 1 row



In [12]:
venta.count()

46645

In [13]:
venta_join_prod = venta.join(producto, producto["ID_PRODUCTO"] == venta["IdProducto"], "left")

In [14]:
venta_join_prod.count()

46645

In [15]:
venta_join_prod.show(1)

+-------+----------+-------------+-------+---------+----------+----------+----------+------+--------+-----------+--------------------+---------+--------+
|IdVenta|     Fecha|Fecha_Entrega|IdCanal|IdCliente|IdSucursal|IdEmpleado|IdProducto|Precio|Cantidad|ID_PRODUCTO|            Concepto|     Tipo|precio_p|
+-------+----------+-------------+-------+---------+----------+----------+----------+------+--------+-----------+--------------------+---------+--------+
|      1|2018-03-09|   2018-03-17|      3|      969|        13|      1674|     42817|813.12|       2|      42817|GAB SENTEY G18-13...|GABINETES|  813.12|
+-------+----------+-------------+-------+---------+----------+----------+----------+------+--------+-----------+--------------------+---------+--------+
only showing top 1 row



In [16]:
venta_join_prod = venta_join_prod.withColumn(
    "PrecioFinal",
    when(venta_join_prod['Precio'] == 0, col("precio_p"))
    .otherwise(col('Precio'))
)

In [17]:
venta_join_prod.show(1)

+-------+----------+-------------+-------+---------+----------+----------+----------+------+--------+-----------+--------------------+---------+--------+-----------+
|IdVenta|     Fecha|Fecha_Entrega|IdCanal|IdCliente|IdSucursal|IdEmpleado|IdProducto|Precio|Cantidad|ID_PRODUCTO|            Concepto|     Tipo|precio_p|PrecioFinal|
+-------+----------+-------------+-------+---------+----------+----------+----------+------+--------+-----------+--------------------+---------+--------+-----------+
|      1|2018-03-09|   2018-03-17|      3|      969|        13|      1674|     42817|813.12|       2|      42817|GAB SENTEY G18-13...|GABINETES|  813.12|     813.12|
+-------+----------+-------------+-------+---------+----------+----------+----------+------+--------+-----------+--------------------+---------+--------+-----------+
only showing top 1 row



In [18]:
# SELECCIONO LAS COLUMNAS 
venta = venta_join_prod.select("IdVenta", "Fecha", "Fecha_Entrega", "IdCanal", "IdCliente",
                               "IdSucursal","IdEmpleado","IdProducto","Cantidad","PrecioFinal")

In [19]:
del venta_join_prod

*** 
Corrijo el codigo de Empleado en una nueva columna cuyo nombre sera tempEmpleado

In [20]:
venta = venta.withColumn("tempEmpleado", (col("IdSucursal") * 1000000) + col("IdEmpleado"))

Borro la columna 'IdEmpleado'

In [21]:
venta = venta.drop("IdEmpleado")

Renombro Columnas

In [22]:
venta = venta.withColumnRenamed("tempEmpleado", "IdEmpleado")\
             .withColumnRenamed("PrecioFinal", "Precio")\
             .withColumnRenamed("Fecha_Entrega", "FechaEntrega")

***
Creo los dataframe `venta_header` y `venta_detail` a partir de `venta`

In [23]:
venta_header = venta.select("IdVenta", "Fecha", "FechaEntrega", "IdCanal", "IdCliente", "IdSucursal", "IdEmpleado")

In [24]:
venta_header.show(1)

+-------+----------+------------+-------+---------+----------+----------+
|IdVenta|     Fecha|FechaEntrega|IdCanal|IdCliente|IdSucursal|IdEmpleado|
+-------+----------+------------+-------+---------+----------+----------+
|      1|2018-03-09|  2018-03-17|      3|      969|        13|  13001674|
+-------+----------+------------+-------+---------+----------+----------+
only showing top 1 row



In [25]:
venta_detail = venta.select("IdVenta", "IdProducto", "Precio", "Cantidad")

In [26]:
venta_detail.show(1)

+-------+----------+------+--------+
|IdVenta|IdProducto|Precio|Cantidad|
+-------+----------+------+--------+
|      1|     42817|813.12|       2|
+-------+----------+------+--------+
only showing top 1 row



In [27]:
del venta

***
Creando la Conexion

In [22]:
engine = connec_db()

***
Pasando los Dataframes a Dataframes de Pandas  y vaciando en sus respectivas Tablas

In [28]:
# IMPORTANDO LIBRERIA PANDAS
import pandas as pd

***
Vaciando el Dataframe a la Tabla `venta_header`

In [29]:
pandas_df_venta_h = venta_header.toPandas()


In [25]:
pandas_df_venta_h.to_sql(name="venta_header", con=engine, index=False, if_exists="append")

46645

In [30]:
pandas_df_venta_h.to_csv("csv_practica/venta_header.csv", index=False)

In [31]:
del pandas_df_venta_h
del venta_header

***
Vaciando el Dataframe a la Tabla `venta_detail`

In [32]:
pandas_df_venta_d = venta_detail.toPandas()

In [ ]:
pandas_df_venta_d.to_sql(name="venta_detail", con=engine, index=False, if_exists="append")

In [34]:
pandas_df_venta_d.to_csv("csv_practica/venta_detail.csv", index=False)

In [35]:
del venta_detail
del pandas_df_venta_d

In [36]:
spark.stop()